**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N253acfc7f53a4ead88d92cbe3236236c (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [3]:
ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s,p,o)

from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)

for r in g.query(q1):
  print(r)

http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [4]:
ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    print(s1)

from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT ?s1
  WHERE { 
    { 
      ?s1 rdf:type ns:Person. 
    } UNION {
      ?s2 rdfs:subClassOf ns:Person.
      ?s1 rdf:type ?s2
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)

for r in g.query(q1):
  print(r)

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [5]:
ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for s1,p1,o1 in g.triples((s, None, None)):
      print(s1,p1,o1)

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    for s2,p2,o2 in g.triples((s1, None, None)):
      print(s2,p2,o2)

from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT ?s ?o ?p
  WHERE { 
    { 
      ?s rdf:type ns:Person.
      ?s ?o ?p
    } UNION {
      ?s rdf:type ?s1.
      ?s ?o ?p.
      ?s1 rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)

for r in g.query(q1):
  print(r)

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#h